<a href="https://colab.research.google.com/github/grace3999/PPs/blob/master/5_FIFI_image_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import os
import requests 
from multiprocessing.pool import ThreadPool
import time
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [3]:
#get list of photo urls corresponding to each FIFI category

#path for FIFI cleaned and preprocessed data frame
path_FIFI = '/content/gdrive/My Drive/WIDS_FIFI_groupproject/generated_datasets/data_final.pkl'

#read in pkl file containing 
data = pd.read_pickle(path_FIFI)
data = pd.DataFrame(data = data)
data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', data.shape, '\n')
print('Original data columns:\n', data.columns.values, '\n')

data = data[data['FIFI_category'] != 'Community Walk']

data['FIFI_category'] = data['FIFI_category'].replace({'Needles/Dumping': 'Needles_Dumping', 'Sign/Signal': 'Sign_Signal'})

#we are only interested in FIFI requests that have a photo included with the request
print(data.shape)
data_photo = data[~data['Photo'].isna()]
print(data_photo.shape)

#create a directory for each category
os.mkdir('/tmp/FIFI_photos/')
save_path = '/tmp/FIFI_photos/'

cats_to_dl = ['Abnd_Vehicle', 'Grafitti', 'Needles_Dumping', 'Parking',
       'Pothole', 'Streetlight', 'Other', 'Clogged_Drain', 'Dead Animal',
       'Vegetation', 'Sign_Signal']

#get and save photos according to FIFI_category
for cat in cats_to_dl:
    if os.path.exists(os.path.join(save_path, cat)) == True:
      print(cat, " already exists")
    else:
      os.mkdir(os.path.join(save_path, cat))
      print(cat, " directory created")

Original data shape:
 (248815, 67) 

Original data columns:
 ['Service_Request_Number' 'Created_Date' 'Location' 'Location_Details'
 'Description' 'License_Number' 'State' 'Vehicle_Make' 'Vehicle_Color'
 'Time_parked' 'Photo' 'FIFI_category' 'Location_Details.1'
 'Property_damage?' 'Current_Issue' 'Type_of_animal'
 'On_private_property?' 'Graffiti_location' 'Location_and_Description'
 'Obscene/Racial/Hateful?' 'Street/Intersection' 'Where?' 'Type_of_item?'
 'Veg_location' 'Vegetation_Safety_Issue' 'Parking_Violation_Concerning'
 'License_Plate_Number' 'Location_details' '7_digit_number_on_pole'
 'Issue' 'date' 'year' 'month' 'day' 'day_of_week' 'hour' 'zip'
 'neighborhood' 'SittingInformal' 'Lying' 'DisruptiveActivity_Aggressive'
 'DisruptiveActivity_Intoxicated' 'LivingActivities' 'Soliciting'
 'overdose' 'pop' 'pop_dens' 'housing_units' 'med_home' 'land_area'
 'occupied_units' 'occupied_ratio' 'med_income' 'med_age' 'male_%'
 'white_%' 'fam_%' 'nokid_%' 'mortgage_%' 'fulltime_%' 'une

In [4]:
#create function to get photo from url and save

def fetch_and_save(image_url_tupple):
    image_name, image_url = image_url_tupple
    #create file name for photo
    image_name = os.path.join(save_path_cat, image_name)  
    #get photo from url and save
    r = requests.get(image_url, allow_redirects=True)
    open(image_name, 'wb').write(r.content)

#do each category individually and 5,000 at a time (runtime keeps dying)
save_path = '/tmp/FIFI_photos/'

cats_to_dl = ['Abnd_Vehicle', 'Grafitti', 'Needles_Dumping', 'Parking',
       'Pothole', 'Streetlight', 'Other', 'Clogged_Drain', 'Dead Animal',
       'Vegetation', 'Sign_Signal']

cat = cats_to_dl[0]
print(cat)
cat_data = data_photo[data_photo['FIFI_category'] == cat].reset_index() #index will get saved as part of file name for later use

save_path_cat = os.path.join(save_path, cat)

i = 0
while i < (5000):
  if i%50 == 0:
    print(i)
  urls = list(zip([cat+'_'+str(x)+'.jpeg' for x in cat_data.index.values[i:i+5]], cat_data['Photo'][i:i+5]))
  ThreadPool(5).imap_unordered(fetch_and_save, urls)
  i += 5

Abnd_Vehicle
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950


In [0]:
#create directories for training and testing 
project_dir = '/tmp/'

try:
  os.mkdir(os.path.join(project_dir, 'cnn_cat_other'))
except OSError:
  pass
try:
  os.mkdir(os.path.join(project_dir, 'cnn_cat_other/training'))
except OSError:
  pass
try:
  os.mkdir(os.path.join(project_dir, 'cnn_cat_other/testing'))
except OSError:
  pass


In [0]:
#create directories for train/test for each FIFI category 
photo_categories = ['Needles_Dumping', 'Grafitti', 'Abnd_Vehicle', 'Parking', 'Pothole', 'Sign_Signal', 'Streetlight']

train_dir = '/tmp/cnn_cat_other/training/'
test_dir = '/tmp/cnn_cat_other/testing/'

for cat in photo_categories:
  try:
    os.mkdir(os.path.join(train_dir, cat))
  except OSError:
   print('cannot make directory for train: ', cat)
  try:
    os.mkdir(os.path.join(test_dir, cat))
  except OSError:
   print('cannot make directory for test: ', cat)

In [0]:
#function to split data into training and testing based on a specified split size
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = os.path.join(SOURCE, filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(TRAINING, filename)
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(TESTING, filename)
        copyfile(this_file, destination)

In [8]:
photos_dir = '/tmp/FIFI_photos/'
train_dir = '/tmp/cnn_cat_other/training/'
test_dir = '/tmp/cnn_cat_other/testing/'

for cat in photo_categories[2:3]:
  source_dir = os.path.join(photos_dir, cat)
  train_dir = os.path.join(train_dir, cat)
  test_dir = os.path.join(test_dir, cat)

  split_size = .9
  start_time = time.time()
  split_data(source_dir, train_dir, test_dir, split_size)
  print("--- %s seconds ---" % (time.time() - start_time))


--- 4.332841634750366 seconds ---
